# Зээлийн өгөгдлийн ангиллын анализ

Энэхүү дүн шинжилгээнд бид `loan_data.csv` файлыг ашиглан зээлийн хэмжээгээр (`Amount`) өндөр эсэхийг ангиллын аргаар тодорхойлж, олон төрлийн ангиллын загваруудыг туршина.

**Алхамууд:**
1. Өгөгдөл унших  
2. Анхан шатны дүн шинжилгээ (EDA)  
3. Ангиллын зорилго (`HighAmount`) үүсгэх  
4. Өгөгдөл бэлтгэл (Шимт цэгшүүлэлт, хуваалт)  
5. Моделийн сургалт (LogisticRegression, DecisionTree, RandomForest, KNN, SVM, NaiveBayes)  
6. Үр дүнг үнэлэх (accuracy, precision, recall, f1, ROC AUC)  


In [1]:
# 1. Шаардлагатай сангуудыг импортлох
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report


In [2]:
# 2. Өгөгдлийг унших
df = pd.read_csv('loan_data.csv')
df.head()

,LoanID,CustomerID,Amount,InterestRate,LoanDate,TermMonths
0,1,115795,14160.61,15.65,2024-10-10,12
1,2,100860,36431.40,11.60,2024-04-30,60
2,3,176820,8808.68,19.76,2020-11-12,60
3,4,154886,41157.59,10.85,2021-10-21,60
4,5,106265,30985.18,8.23,2023-04-08,12


In [3]:
# 3. Ангиллын зорилго үүсгэх: 
# 'HighAmount' = Amount > median(Amount)
median_amount = df['Amount'].median()
df['HighAmount'] = (df['Amount'] > median_amount).astype(int)
df['HighAmount'].value_counts()

HighAmount
0    50000
1    50000
Name: count, dtype: int64

In [4]:
# 4. Өгөгдөл бэлтгэл: 
# Шаардлагатай шинж чанарыг сонгох
features = ['InterestRate', 'TermMonths']
X = df[features]
y = df['HighAmount']

# Мэдээллийг сургалт, шалгалтын бүлэгт хуваах
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Шимт (scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# 5. Моделүүдийг тодорхойлох ба сургалт (бүх модельд нэгэн үйлдэлтэйгээр)
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine': SVC(probability=True, random_state=42),
    'Naive Bayes': GaussianNB()
}

results = []

for name, model in models.items():
    # Сургалт
    model.fit(X_train_scaled, y_train)
    # Таамаглал
    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]
    # Үнэлгээ
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)
    results.append({
        'Model': name,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROC AUC': roc_auc
    })

# DataFrame болгон харуулах
results_df = pd.DataFrame(results).sort_values(by='ROC AUC', ascending=False)
results_df